In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

data = fetch_california_housing()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
import torch

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

In [7]:
import torch.nn as nn
import torch.optim as optim

epochs = 1500
batch_size = 64

class CaliforniaHousingModel(nn.Module):
    def __init__(self):
        super(CaliforniaHousingModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(8, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ELU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Dropout(0.1),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        
    def forward(self, x):
        return self.network(x)

model = CaliforniaHousingModel().to(device)

criterion = nn.SmoothL1Loss() 
optimizer = optim.RMSprop(model.parameters(), lr=0.001, weight_decay=1e-5)

In [8]:
import torch.nn as nn
import torch.optim as optim

def mean_absolute_percentage_error(y_true, y_pred):
    return torch.mean(torch.abs((y_true - y_pred) / y_true)).item()

for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(X_train_tensor.size(0))
    epoch_loss = 0.0

    for i in range(0, X_train_tensor.size(0), batch_size):
        indices = permutation[i:i + batch_size]
        batch_x, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    with torch.no_grad():
        model.eval()
        y_pred = model(X_test_tensor)
        mse = criterion(y_pred, y_test_tensor).item()
        mape = mean_absolute_percentage_error(y_test_tensor, y_pred)

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(X_train_tensor):.4f}")

print("\nModel Evaluation:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.4f}")

Epoch 50/1500, Loss: 0.0018
Epoch 100/1500, Loss: 0.0016
Epoch 150/1500, Loss: 0.0015
Epoch 200/1500, Loss: 0.0014
Epoch 250/1500, Loss: 0.0014
Epoch 300/1500, Loss: 0.0014
Epoch 350/1500, Loss: 0.0013
Epoch 400/1500, Loss: 0.0013
Epoch 450/1500, Loss: 0.0013
Epoch 500/1500, Loss: 0.0013
Epoch 550/1500, Loss: 0.0012
Epoch 600/1500, Loss: 0.0012
Epoch 650/1500, Loss: 0.0012
Epoch 700/1500, Loss: 0.0012
Epoch 750/1500, Loss: 0.0012
Epoch 800/1500, Loss: 0.0012
Epoch 850/1500, Loss: 0.0012
Epoch 900/1500, Loss: 0.0012
Epoch 950/1500, Loss: 0.0011
Epoch 1000/1500, Loss: 0.0012
Epoch 1050/1500, Loss: 0.0011
Epoch 1100/1500, Loss: 0.0012
Epoch 1150/1500, Loss: 0.0011
Epoch 1200/1500, Loss: 0.0011
Epoch 1250/1500, Loss: 0.0011
Epoch 1300/1500, Loss: 0.0011
Epoch 1350/1500, Loss: 0.0011
Epoch 1400/1500, Loss: 0.0011
Epoch 1450/1500, Loss: 0.0011
Epoch 1500/1500, Loss: 0.0011

Model Evaluation:
Mean Squared Error (MSE): 0.1042
Mean Absolute Percentage Error (MAPE): 0.1785
